In [56]:
# Librairies à importer pour utiliser JuMP avec le solver GLPK
using JuMP
using GLPK

# Définition de constantes pour le statut de résolution du problème
const OPTIMAL = JuMP.MathOptInterface.OPTIMAL
const INFEASIBLE = JuMP.MathOptInterface.INFEASIBLE
const UNBOUNDED = JuMP.MathOptInterface.DUAL_INFEASIBLE;

#V est le tableau qui contient la liste des sommets du graph par leurs positions x, y
#S est le teableau qui contient le type du sommet : 
#    0 : sommet lambda
#    1 : sommet obligatoire
#    2 : sommet de depart
#    3 : sommet d'arrivée
#G et la matrice d'adjacence du graph

#la function resolve trouve le chemin le plus court sous la forme d'une matrice d'adjacence (orientée)

function resolve(V, S, G)
    
    #hauteur et largeur de la matrice d'adjacence
    x = length(G[:,1])
    y = length(G[1,:])
    
    #distance entre les sommets
    Distance = zeros(Float64, x, y)
    for i in 1:x
        for j in 1:y
            if (G[i, j] == 0)
                Distance[i, j] = 0
            else    
                Distance[i, j] = hypot(V[i, 1] - V[j, 1], V[i, 2] - V[j, 2])
            end
        end
    end
    
    nbr = 0
    Vstart = 1
    Vend = 1
    for i in 1:x
        if (S[i] == 1)
            nbr = nbr + 1
        end
        if (S[i] == 2)
            Vstart = i
        end
        if (S[i] == 3)
            Vend = i
        end
    end
    
    # Création d'un modèle. Ce modèle fera l'interface avec le solveur GLPK
    m = Model(with_optimizer(GLPK.Optimizer))

    # Création des variables : matrice d'adjacence du sous graphe représentant le chemin (graph orienté)
    @variable(m, R[1:y,1:x] >= 0, Int)
    # Création des variables servant à l'élimination des sous-tours
    @variable(m, Z[1:y,1:x] >= 0, Int)
   
    # Fonction objectif
    @objective(m, Min, sum(R .* Distance))
   
    #Création des contraintes
    
    #existence des aretes utilisées (ou plutot non utilisation de celles qui n'existent pas) et
    #pas le droit de retourner en arriere
    for i in 1:x
        for j in 1:y
            if (G[j, i] == 0)
                @constraint(m, R[j, i] == 0)    
            elseif (i > j)
                @constraint(m, R[j, i] + R[i, j] <= 1)
            end
        end
    end

    
    for i in 1:x
        #sommet de départ : on doit avoir une arete qui en part de plus que le nombre d'arete qui y rentre
        if (S[i] == 2)
            @constraint(m, sum(R[i, :]) == 1)
            @constraint(m, sum(R[:, i]) == 0)
        #sommet d'arrivée : on doit avoir une arete qui en part de plus que le nombre d'arete qui y rentre
        elseif (S[i] == 3)
            @constraint(m, sum(R[:, i]) == 1)
            @constraint(m, sum(R[i, :]) == 0)
        #sommets obligatoire : on doit avoir autant de sommet qui partent que de somemts qui arrivent
        # mais au moins un vu qu'il est obligatoire
        elseif (S[i] == 1)
            @constraint(m, sum(R[:, i]) == 1)
            @constraint(m, sum(R[i, :]) == 1)
        #sommets random : on doit avoir autant de sommet qui partent que de somemts qui arrivent
        elseif (S[i] == 0)
            @constraint(m, sum(R[i, :] - R[:, i]) == 0)
        end
    end

    #contraintes enmpechant la formation de sous-tours
 
    for i in 1:x
        if (S[i] == 2)
            @constraint(m, sum(Z[i, :]) == nbr + 1)
        end
        if (S[i] == 3)
            @constraint(m, sum(Z[:, i]) == 1)
        end
        if (S[i] == 1)
            @constraint(m, sum(Z[i, :]) + 1 == sum(Z[:, i]))
        end
        if (S[i] == 0)
            @constraint(m, sum(Z[i, :]) == sum(Z[:, i]))
        end
    end
    
    for i in 1:x
        for j in 1:y
            if (i != j && j != Vstart)
                @constraint(m, Z[i, j] <= (nbr+1) * (R[i, j]))
            end
        end
    end
    
    #Affichage du modèle
    #print(m)

    #Résolution du problème d'optimisation linéaire m par le solveur GLPK
    optimize!(m)

    status = termination_status(m)

    if status == INFEASIBLE
        println("Le problème n'est pas réalisable")
    elseif status == UNBOUNDED
        println("Le problème est non borné")
    elseif status == OPTIMAL
        println("Optimum = ", JuMP.objective_value(m))
        println("Solution optimale :")
        for i in 1:x
            for j in 1:y
               print(JuMP.value(R[i, j]), " ")
            end
            println()
        end
#        println("Flo :")
#        for i in 1:x
#           for j in 1:y
#              print(JuMP.value(Z[i, j]), " ")
#          end
#           println()
#       end
    else
        println("Problème lors de la résolution")
    end
end

resolve (generic function with 1 method)

In [57]:
V = [0.0 0.0; 0.0 4.0; 2.0 2.0; 4.0 0.0; 4.0 4.0]
S = [2 1 0 0 3]
G = [0 1 1 1 0;
     1 0 1 0 1;
     1 1 0 1 1;
     1 0 1 0 1;
     0 1 1 1 0]
resolve(V, S, G)

Optimum = 8.0
Solution optimale :
0.0 1.0 0.0 0.0 0.0 
0.0 0.0 0.0 0.0 1.0 
0.0 0.0 0.0 0.0 0.0 
0.0 0.0 0.0 0.0 0.0 
0.0 0.0 0.0 0.0 0.0 


In [58]:
V = [0.0 0.0; 0.0 4.0; 2.0 2.0; 4.0 0.0; 4.0 4.0]
S = [2 0 0 0 3]
G = [0 1 1 1 0;
     1 0 1 0 1;
     1 1 0 1 1;
     1 0 1 0 1;
     0 1 1 1 0]
resolve(V, S, G)

Optimum = 5.656854249492381
Solution optimale :
0.0 0.0 1.0 0.0 0.0 
0.0 0.0 0.0 0.0 0.0 
0.0 0.0 0.0 0.0 1.0 
0.0 0.0 0.0 0.0 0.0 
0.0 0.0 0.0 0.0 0.0 


In [59]:
V = [0.0 0.0; 1.0 1.0; 3.0 3.0; 5.0 5.0; 6.0 6.0]
S = [2 0 0 0 3]
G = [0 1 0 0 0;
     1 0 1 0 0;
     0 1 0 1 0;
     0 0 1 0 1;
     0 0 0 1 0]
resolve(V, S, G)

Optimum = 8.485281374238571
Solution optimale :
0.0 1.0 0.0 0.0 0.0 
0.0 0.0 1.0 0.0 0.0 
0.0 0.0 0.0 1.0 0.0 
0.0 0.0 0.0 0.0 1.0 
0.0 0.0 0.0 0.0 0.0 


In [60]:
V = [202.0 0.0; 200.0 0.0; 2.0 0.0; 0.0 0.0; 1.0 1.0; 201.0 1.0]
S = [0 1 0 3 2 1]
G = [0 1 0 0 0 1;
     1 0 1 0 0 1;
     0 1 0 1 1 0;
     0 0 1 0 1 0;
     0 0 1 1 0 1;
     1 1 0 0 1 0]
resolve(V, S, G)

Optimum = 401.4142135623731
Solution optimale :
0.0 0.0 0.0 0.0 0.0 0.0 
0.0 0.0 1.0 0.0 0.0 0.0 
0.0 0.0 0.0 1.0 0.0 0.0 
0.0 0.0 0.0 0.0 0.0 0.0 
0.0 0.0 0.0 0.0 0.0 1.0 
0.0 1.0 0.0 0.0 0.0 0.0 
